In [1]:
!pip install confluent-kafka yfinance minio pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 4.4 MB/s eta 0:00:00a 0:00:010m
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 7.5 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.6/84.6 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 11.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.6 MB/s eta 0:00:0000:0100:01
  Created wheel for peewee: filename=peewee-3.17.6-cp311-cp311-linux_x86_64.whl size=274819 sha256=2f83cf4fc3068e6f9fac0ac8d67d0790a52480ec14f38303ec1c8962f21555e8
  Stored in directory: /home/jovyan/.cache/pip/wheels/1c/09/7e/9f659fde248ecdc1722a142c1d744271aad3914a0afc191058
S

In [6]:
import yfinance as yf
import pandas as pd

def fetch_stock_data(ticker, start_date, end_date):
    stock = yf.Ticker(ticker)
    data = stock.history(start=start_date, end=end_date)
    return data

In [7]:
from confluent_kafka import Producer
import json

def send_data_to_kafka(topic, data):
    conf = {'bootstrap.servers': 'kafka:9092'}  # Replace with your Kafka broker addresses
    producer = Producer(conf)

    def delivery_report(err, msg):
        if err is not None:
            print(f"Delivery failed: {err}")
        else:
            print(f"Message delivered to {msg.topic()} [{msg.partition()}]")
            
    for index, row in data.iterrows():
        # Convert the row to a dictionary
        message = row.to_dict()
        
        # Convert any Timestamp objects to strings
        for key, value in message.items():
            if isinstance(value, pd.Timestamp):
                message[key] = value.isoformat()
        
        # Serialize the message to JSON and send it to Kafka
        producer.produce(topic, json.dumps(message), callback=delivery_report)
        producer.poll(1)

    producer.flush()


In [8]:
ticker = 'AAPL'
start_date = '2023-01-01'
end_date = '2023-12-31'
data = fetch_stock_data(ticker, start_date, end_date)

In [9]:
# Reset the index to convert Date from index to a column
data = data.reset_index()
# Rename the new index column to 'Date'
data.rename(columns={'index': 'Date'}, inplace=True)
data

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2023-01-03 00:00:00-05:00,129.066078,129.680297,123.013009,123.904625,112117500,0.0,0.0
1,2023-01-04 00:00:00-05:00,125.707670,127.461182,123.914537,125.182610,89113600,0.0,0.0
2,2023-01-05 00:00:00-05:00,125.945435,126.579471,123.597523,123.855095,80962700,0.0,0.0
3,2023-01-06 00:00:00-05:00,124.835861,129.075971,123.726294,128.412216,87754700,0.0,0.0
4,2023-01-09 00:00:00-05:00,129.254312,132.166920,128.679714,128.937286,70790800,0.0,0.0
...,...,...,...,...,...,...,...,...
245,2023-12-22 00:00:00-05:00,194.442163,194.671304,192.240525,192.868149,37122800,0.0,0.0
246,2023-12-26 00:00:00-05:00,192.878102,193.157042,192.101051,192.320221,28919300,0.0,0.0
247,2023-12-27 00:00:00-05:00,191.762337,192.768513,190.367620,192.419830,48087700,0.0,0.0
248,2023-12-28 00:00:00-05:00,193.406086,193.924125,192.439752,192.848206,34049900,0.0,0.0


In [10]:
data.index

RangeIndex(start=0, stop=250, step=1)

In [12]:
send_data_to_kafka(f'{ticker}_stock', data)

Message delivered to AAPL_stock [0]
Message delivered to AAPL_stock [0]
Message delivered to AAPL_stock [0]
Message delivered to AAPL_stock [0]
Message delivered to AAPL_stock [0]
Message delivered to AAPL_stock [0]
Message delivered to AAPL_stock [0]
Message delivered to AAPL_stock [0]
Message delivered to AAPL_stock [0]
Message delivered to AAPL_stock [0]
Message delivered to AAPL_stock [0]
Message delivered to AAPL_stock [0]
Message delivered to AAPL_stock [0]
Message delivered to AAPL_stock [0]
Message delivered to AAPL_stock [0]
Message delivered to AAPL_stock [0]
Message delivered to AAPL_stock [0]
Message delivered to AAPL_stock [0]
Message delivered to AAPL_stock [0]
Message delivered to AAPL_stock [0]
Message delivered to AAPL_stock [0]
Message delivered to AAPL_stock [0]
Message delivered to AAPL_stock [0]
Message delivered to AAPL_stock [0]
Message delivered to AAPL_stock [0]
Message delivered to AAPL_stock [0]
Message delivered to AAPL_stock [0]
Message delivered to AAPL_st